In [ ]:
import logging

from pydub import AudioSegment

In [ ]:
in_file = "../data/to_extract/AG 2025 mg.m4a"
out_dir = "../data/extracted/"

In [ ]:
audio = AudioSegment.from_file(in_file)

In [ ]:
from pydub import AudioSegment, silence

res = silence.detect_silence(
    audio, min_silence_len=1000, silence_thresh=-50, seek_step=100
)

In [ ]:
res

In [ ]:
for i, silence_range in enumerate(res):
    silence_1 = audio[res[i][0] : res[i][1]]
    silence_1.export(out_dir + f"silence_{i}.wav", format="wav")

In [ ]:
BATCH_SIZE_MS = 5 * 60 * 1000  # 10 minutes
audios = []
batch_start = 0

for silence_start, silence_end in res:
    if silence_end - batch_start > BATCH_SIZE_MS:
        audios.append(audio[batch_start:silence_start])
        batch_start = silence_start


audios.append(audio[batch_start:])

In [ ]:
for i, splitted_audio in enumerate(audios):
    splitted_audio.export(out_dir + f"splitted_audio_{i}.wav", format="wav")

In [ ]:
import os

import requests
from dotenv import load_dotenv

load_dotenv()

PRODUCT_ID = os.getenv("IK_PRODUCT_ID")
OPENAI_API = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai"
IK_API = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}"
LLM_TOKEN = os.getenv("IK_API_KEY")
TRANSCRIPTION_URL = f"{OPENAI_API}/audio/transcriptions"
RESULT_URL = f"{IK_API}/results"
TRANSCRIPTION_HEADERS = {"Authorization": f"Bearer {LLM_TOKEN}"}


def get_batch_id_audio(f_bytes):
    data = {
        "model": "whisper",
        "response_format": "vtt",
        # "max_line_width": max_line_width,
        # "max_line_count": max_line_count,
    }
    files = [("file", ("filename", f_bytes, "audio/wav"))]

    response = requests.request(
        "POST",
        url=TRANSCRIPTION_URL,
        headers=TRANSCRIPTION_HEADERS,
        data=data,
        files=files,
    )
    response.raise_for_status()

    batch_id = response.json().get("batch_id")
    if not batch_id:
        raise ValueError("No batch_id returned from the API.")

    return batch_id

In [ ]:
batch_ids = []
for i, splitted_audio in enumerate(audios):
    batch_ids.append(get_batch_id_audio(splitted_audio.export(format="wav").read()))

In [ ]:
def get_result(batch_id):
    response = requests.request(
        "GET",
        url=f"{RESULT_URL}/{batch_id}",
        headers=TRANSCRIPTION_HEADERS,
    )
    response.raise_for_status()

    result = response.json()

    if result["status"] == "pending":
        return {"status": "PENDING"}
    if result["status"] == "success":
        return {"status": "SUCCESS", "result": result["data"]}
    if result["status"] == "error":
        return {"status": "FAILURE", "error": "Server error"}

    return {
        "status": "error",
        "error": f"Unknown status from the server {result['status']}",
    }

In [ ]:
text = ""

for batch_id in batch_ids:
    result = get_result(batch_id)
    if result["status"] == "SUCCESS":
        text += result["result"]
    elif result["status"] == "PENDING":
        logging.info(f"Waiting for results: {result}")
    else:
        logging.error(f"Error for result: {result}")

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv("IK_API_KEY")
PRODUCT_ID = os.getenv("IK_PRODUCT_ID")
URL = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai"
MODEL_LLM = "mistral24b"

from openai import OpenAI

llm_base_client = OpenAI(
    api_key=TOKEN,
    base_url=URL,
)

In [ ]:
print(text)

In [ ]:
existing_summary = """
# Magic Genève AG 2025
#mtg

Date: 21.03.2025
Lieu: Espace de quartier Clos Voltaire

### Liste de présence:
* François Matthey
* Christophe Hostetter
* Elliott
* Miko
* Michal
* Timothé
* Lucas
* Alexi
* Nicolas Casademont
* Mark Schwass
* Yannick
* Engin Paca 

**Ouverture de séance 18h10**

### Définition de responsables par groupement de taches
* Responsables tournois DC
  * François Matthey
  * Elliott
* Responsables tournois Legacy
  * Christophe
* Responsables tournois Modern
  * Christophe
  * Lucas
* Responsables tournois Standard
  * Alexi
  * Nicolas
* Responsables tournois Limités
  * Mark
  * Eliott
  * Nicolas
* Responsables Commander Multi
  * Engin Paca
* Responsables livraisons xénomorphes
  * Mark
  * Yannick
* Responsable Calendrier
  * Nicolas
* Responsables ouverture des portes
  * Alexi
  * Mark
  * Elliot
  * Nicolas
* Responsables site internet
  * Nicolas
  * Michal
* Responables Marketing
  * Michal
  * Eliott
  * Timothé
* Responsables Discord
  * Alexi


### Membres du comité 2025
**Président**: François
**Trésorier**: Yannick (uniquement finalisation des comptes)
**Vices trésoriers**: Lucas, Christophe (Garantissent que les fiches soient remplies à la fin de la semaine)
**Secrétaire**: Nicolas
**Membres du comités**:
* François
* Yannick
* Lucas
* Christophe
* Nicolas
* Eliott
* Michal
* Alexi
* Engin Paca
* Mark
* Alain ?
**Départs**:
* Miko
* Thomas


---
### Prise de note Yannick

**François :**

- Bénéfice 2024 : 3819.5 €
- Rapport de trésorerie doit passer par Lucas Piot
- Non encore validé à ce jour
- Erreur possible sur comptabilité mais non significatif (problème sur certains événements, commissions twint)
- Nom du domaine sur Infomaniak
- Rapport financier sur les entrées de CA et dépenses
- Proposition de répartition des tâches de l'association/organisation
- Appel à volontaires selon liste jointe : pour organisation
- Vote pour le comité : ajout d'un nouveau poste de Vice-Trésorier
- Rappel des rôles des membres du comité

**Départs :** Miko, Thomas
5 membres se représentent
5 nouveaux candidats
Tous les candidats sont élus à l'unanimité

**Description des rôles des membres du bureau :**

- **Secrétariat :** prendre les PV des réunions et rédiger des e-mails aux membres -> Nicolas Voté à l’unanimité
  - Envoyer des e-mails aux membres
  - Courriels divers (p. ex. : réponse plainte salle)

* **Trésoriers**: 
  - Tenue des comptes
  - Gérer le paiement des lots et des achats
  - Yannick trésorier, voté à l’unanimité
  - Yannick ne fera pas le quotidien mais validera les comptes en fin d’année
  - Christophe et Lucas Vice Trésoriés,  voté à l’unanimité
  - Christophe et Lucas gérerons le quotidien de la trésorerie

- **Président :** responsable légal de l'association
  - François : élu à l'unanimité

**Questions diverses :**

- Manque de personnes pour les drafts
* Plus de pubs (instagram, tik tok, etc…) -> création de responsable marketing
- Organiser d'autres jeux pour faire venir d'autres personnes
- Image de l'association parfoit trop compétitif
- Tenter de faire 2X AP
- 54 membres en 2024

---
"""

In [ ]:
response_stream = llm_base_client.chat.completions.create(
    model=MODEL_LLM,
    messages=[
        {
            "role": "user",
            "content": f"Rédige moi un PV de l'AG 2025 de l'association magic Genève basé à la fois sur la restranscription de l'audio: \n\n{text}\n\n. Ainsi que quelques notes déjà prises: \n\n{existing_summary}\n\n. N'invente rien.",
        }
    ],
    temperature=0.3,
    stream=True,
)
summary = ""
# Stream the response
for response in response_stream:
    print(response.choices[0].delta.content or "", end="", flush=True)
    summary += response.choices[0].delta.content or ""